# Tutorial on Hand Written Digit Recognition

In this tutorial we will go through the basic use case of MXNet and also touch on some advanced usages. This example is based on the MNIST dataset, which contains 70,000 images of hand written characters with 28-by-28 pixel size.

This tutorial covers the following topics:
- network definition.
- Variable naming.
- Basic data loading and training with feed-forward deep neural networks.
- Monitoring intermediate outputs for debuging.
- Custom training loop for advanced models.

Let’s train a 3-layer multilayer perceptron on the MNIST dataset to classify handwritten digits. 

First, let's load mxnet scala jar in classpath.

## Jupyter Scala kernel
Add mxnet scala jar which is created as a part of MXNet Scala package installation in classpath as follows:

**Note**: Process to add this jar in your scala kernel classpath can differ according to the scala kernel you are using.

We have used [jupyter-scala kernel](https://github.com/alexarchambault/jupyter-scala) for creating this notebook.

```
classpath.addPath(<path_to_jar>)

e.g
classpath.addPath("mxnet-full_2.11-osx-x86_64-cpu-0.1.2-SNAPSHOT.jar")
```



import required modules

In [2]:
import ml.dmlc.mxnet._
import ml.dmlc.mxnet.optimizer.SGD
import scala.collection.mutable.ListBuffer

import ml.dmlc.mxnet._
import ml.dmlc.mxnet.optimizer.SGD
import scala.collection.mutable.ListBuffer

## Network Definition

Now, we can start constructing our network:


In [3]:
// # Variables are place holders for input arrays. We give each variable a unique name.
val data = Symbol.Variable("data")

// The input is fed to a fully connected layer that computes Y=WX+b.
// This is the main computation module in the network.
// Each layer also needs an unique name. We'll talk more about naming in the next section.
val fc1 = Symbol.FullyConnected(name = "fc1")()(Map("data" -> data, "num_hidden" -> 128))

// Activation layers apply a non-linear function on the previous layer's output.
// Here we use Rectified Linear Unit (ReLU) that computes Y = max(X, 0).
val act1 = Symbol.Activation(name = "relu1")()(Map("data" -> fc1, "act_type" -> "relu"))

val fc2 = Symbol.FullyConnected(name = "fc2")()(Map("data" -> act1, "num_hidden" -> 64))
val act2 = Symbol.Activation(name = "relu2")()(Map("data" -> fc2, "act_type" -> "relu"))
val fc3 = Symbol.FullyConnected(name = "fc3")()(Map("data" -> act2, "num_hidden" -> 10))

// Finally we have a loss layer that compares the network's output with label and generates gradient signals.
val mlp = Symbol.SoftmaxOutput(name = "softmax")()(Map("data" -> fc3))

log4j:WARN No appenders could be found for logger (MXNetJVM).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


data: Symbol = ml.dmlc.mxnet.Symbol@2aec6c99
fc1: Symbol = ml.dmlc.mxnet.Symbol@60bcbe48
act1: Symbol = ml.dmlc.mxnet.Symbol@60abab94
fc2: Symbol = ml.dmlc.mxnet.Symbol@2665643d
act2: Symbol = ml.dmlc.mxnet.Symbol@20b63589
fc3: Symbol = ml.dmlc.mxnet.Symbol@25fd8129
mlp: Symbol = ml.dmlc.mxnet.Symbol@18b77909

## Variable Naming
MXNet requires variable names to follow certain conventions:
- All input arrays have a name. This includes inputs (data & label) and model parameters (weight, bias, etc).
- Arrays can be renamed by creating named variable. Otherwise, a default name is given as 'SymbolName_ArrayName'. For example, FullyConnected symbol fc1's weight array is named as 'fc1_weight'.
- Although you can also rename weight arrays with variables, weight array's name should always end with '_weight' and bias array '_bias'. MXNet relies on the suffixes of array names to correctly initialize & update them.
Call listArguments method on a symbol to get the names of all its inputs:

In [4]:
mlp.listArguments()

res3: IndexedSeq[String] = ArrayBuffer(
  "data",
  "fc1_weight",
  "fc1_bias",
  "fc2_weight",
  "fc2_bias",
  "fc3_weight",
  "fc3_bias",
  "softmax_label"
)

## Data Loading

We download the MNIST data using the [get_mnist_data script](https://github.com/dmlc/mxnet/blob/master/scala-package/core/scripts/get_mnist_data.sh). Now we can create data iterators from our MNIST data. A data iterator returns a batch of data examples each time for the network to process. MXNet provide a suite of basic DataIters for parsing different data format. 

Here we use MNISTIter, which wraps around a numpy array and each time slice a chunk from it along the first dimension. 
Change path of input files according to your system.
Load the training and validation data using DataIterators as follows:



In [5]:
// load MNIST dataset
val trainDataIter = IO.MNISTIter(Map(
  "image" -> "data/train-images-idx3-ubyte",
  "label" -> "data/train-labels-idx1-ubyte",
  "data_shape" -> "(1, 28, 28)",
  "label_name" -> "softmax_label",
  "batch_size" -> "50",
  "shuffle" -> "1",
  "flat" -> "0",
  "silent" -> "0",
  "seed" -> "10"))

val valDataIter = IO.MNISTIter(Map(
  "image" -> "data/t10k-images-idx3-ubyte",
  "label" -> "data/t10k-labels-idx1-ubyte",
  "data_shape" -> "(1, 28, 28)",
  "label_name" -> "softmax_label",
  "batch_size" -> "50",
  "shuffle" -> "1",
  "flat" -> "0", "silent" -> "0"))


trainDataIter: DataIter = non-empty iterator
valDataIter: DataIter = non-empty iterator

## Training
With the network and data source defined, we can finally start to train our model. We do this with MXNet's convenience wrapper for FeedForward builder.


In [6]:
// setup model and fit the training data
val model = FeedForward.newBuilder(mlp) // Use the network we just defined
      .setContext(Context.cpu()) // Run on CPU 
      .setNumEpoch(10) // Train for 10 epochs
      .setOptimizer(new SGD(learningRate = 0.1f, momentum = 0.9f, wd = 0.0001f)) // Learning rate, 
//Momentum and Weight decay for regularization
      .setTrainData(trainDataIter) // Training data set
      .setEvalData(valDataIter) // Testing data set. MXNet computes scores on test set every epoch
      .build()


model: FeedForward = ml.dmlc.mxnet.FeedForward@3d40f3b0

## Evaluation¶

After the model is trained, we can evaluate it on a held out validation dataset and compare the predicted labels with the real labels.

In [7]:
val probArrays = model.predict(valDataIter)
// in this case, we do not have multiple outputs
require(probArrays.length == 1)
val prob = probArrays(0)

// get real labels
valDataIter.reset()
val labels = ListBuffer.empty[NDArray]
while (valDataIter.hasNext) {
  val evalData = valDataIter.next()
  labels += evalData.label(0).copy()
}
val y = NDArray.concatenate(labels)

// get predicted labels
val predictedY = NDArray.argmax_channel(prob)
require(y.shape == predictedY.shape)


probArrays: Array[NDArray] = Array(ml.dmlc.mxnet.NDArray@7d18083e)
prob: NDArray = ml.dmlc.mxnet.NDArray@426c4e40
labels: ListBuffer[NDArray] = ListBuffer(
  ml.dmlc.mxnet.NDArray@2accfeb0,
  ml.dmlc.mxnet.NDArray@5f74e322,
  ml.dmlc.mxnet.NDArray@63792053,
  ml.dmlc.mxnet.NDArray@3ed497c6,
  ml.dmlc.mxnet.NDArray@51274f83,
  ml.dmlc.mxnet.NDArray@4a0bed53,
  ml.dmlc.mxnet.NDArray@476a7696,
  ml.dmlc.mxnet.NDArray@3370c62e,
  ml.dmlc.mxnet.NDArray@32e012b4,
  ml.dmlc.mxnet.NDArray@d1db10e,
  ml.dmlc.mxnet.NDArray@2b949b3e,
  ml.dmlc.mxnet.NDArray@43f2a5f5,
  ml.dmlc.mxnet.NDArray@27a485dd,
  ml.dmlc.mxnet.NDArray@21c6cebf,
  ml.dmlc.mxnet.NDArray@30d3eb9f,
  ml.dmlc.mxnet.NDArray@1932251f,
  ml.dmlc.mxnet.NDArray@47b7adba,
  ml.dmlc.mxnet.NDArray@22e85214,
  ml.dmlc.mxnet.NDArray@1f29cb21,
...
y: NDArray = ml.dmlc.mxnet.NDArray@76d55afb
predictedY: NDArrayFuncReturn = ml.dmlc.mxnet.NDArrayFuncReturn@4786ef4d

We can also evaluate the model's accuracy on the entire test set:

In [8]:
// calculate accuracy
var numCorrect = 0
var numTotal = 0
for ((labelElem, predElem) <- y.toArray zip predictedY.toArray) {
  if (labelElem == predElem) {
    numCorrect += 1
  }
  numTotal += 1
}
val acc = numCorrect.toFloat / numTotal
println(s"Final accuracy = $acc")


Final accuracy = 0.9609


numCorrect: Int = 9609
numTotal: Int = 10000
acc: Float = 0.9609F

## Next Steps¶
Check out more MXNet Scala resources below.

[Scala API](http://mxnet.io/api/scala/)

[More Scala Examples](https://github.com/dmlc/mxnet/tree/master/scala-package/examples/src/main/scala/ml/dmlc/mxnet/examples)

[MXNet tutorials index](http://mxnet.io/tutorials/index.html)